<a href="https://colab.research.google.com/github/Luc4P4nt1/Paraphrazer/blob/main/Paraphraser_v10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-

# VERSIONE v4.13.2-MD_TABLESPACE_POSTPROC_LAYOUT:
# - AGGIUNTA post-elaborazione per forzare righe vuote prima/dopo tabelle Markdown.
# - Mantiene: Analisi Layout Img Pos, Fix BBox tuple, Fix spazio tabelle (prompt), No Hr/Pg/Tl, Logo fisso, Modello originale, etc.
# Basato su v4.13.1

# --- 0. Installazione Librerie ---
!pip install -q google-generativeai pymupdf google-colab Pillow

# --- 1. Importazioni ---
import google.generativeai as genai
from google.colab import userdata, files, drive
import fitz  # PyMuPDF
from fitz import Rect
import io
import time
import pathlib
import textwrap
import os
import re
import sys
import traceback
import math
import unicodedata
from PIL import Image, UnidentifiedImageError
import shutil

# --- CONFIGURAZIONE UTENTE ---
PERMANENT_HEADER_LOGO_SOURCE_PATH = "/content/drive/MyDrive/Appunti TopNotes/logo/logo.jpg"
# !!! ASSICURATI CHE QUESTO FILE ESISTA NELLA POSIZIONE SPECIFICATA !!!
# ---------------------------

# --- 2. Configurazione API Key Gemini ---
# (Invariato)
try:
    GOOGLE_API_KEY = userdata.get('GEMINI_API_KEY')
    if not GOOGLE_API_KEY: raise ValueError("API Key non fornita o non trovata nei Secret.")
    genai.configure(api_key=GOOGLE_API_KEY)
    print("API Key Gemini caricata con successo.")
except (userdata.SecretNotFoundError, ValueError) as ve: print(f"Errore Critico: Chiave API 'GEMINI_API_KEY' non trovata. {ve}"); sys.exit(1)
except Exception as e: print(f"Errore Critico Configurazione API: {e}"); sys.exit(1)

# --- 3. Impostazioni Modello Gemini ---
# (Invariato)
MODEL_NAME_REQUESTED = 'gemini-2.5-flash-preview-04-17'
MODEL_FALLBACK_1 = 'gemini-1.5-pro-latest'
MODEL_FALLBACK_2 = 'gemini-1.0-pro'
generation_config = { "temperature": 0.6, "top_p": 0.95, "top_k": 64, "max_output_tokens": 60000, "response_mime_type": "text/plain" }
safety_settings = [{"category": f"HARM_CATEGORY_{cat}", "threshold": "BLOCK_MEDIUM_AND_ABOVE"} for cat in ["HARASSMENT", "HATE_SPEECH", "SEXUALLY_EXPLICIT", "DANGEROUS_CONTENT"]]
model = None; model_name_in_use = None; model_candidates = [MODEL_NAME_REQUESTED, MODEL_FALLBACK_1, MODEL_FALLBACK_2]
print(f"Modello richiesto: {MODEL_NAME_REQUESTED}"); print(f"Configurazione generazione: {generation_config}")
for model_candidate in model_candidates:
    print(f"Tentativo inizializzazione modello: {model_candidate}")
    try: model = genai.GenerativeModel(model_name=model_candidate, safety_settings=safety_settings, generation_config=generation_config); model_name_in_use = model_candidate; print(f"Modello '{model_name_in_use}' inizializzato con successo."); break
    except Exception as e: print(f"Errore inizializzazione '{model_candidate}': {e}")
if model is None: print("Errore Critico: Nessun modello Gemini inizializzato."); sys.exit(1)


# --- 4. Istruzioni Dettagliate per la Parafrasi ---
# (Invariato da v4.13.1 - Manteniamo l'istruzione per le tabelle come best practice)
PARAPHRASING_INSTRUCTIONS = """
Elabora il testo fornito seguendo rigorosamente queste istruzioni. Genera l'output ESCLUSIVAMENTE in formato MARKDOWN.

Parafrasi del testo:
1.  Riscrivi il testo fornito usando sinonimi, riformulazioni e diverse costruzioni grammaticali, conservando INTEGRALMENTE il significato originale e tutte le informazioni chiave.
2.  Mantieni ASSOLUTAMENTE INVARIATI: nomi propri, acronimi, sigle, riferimenti normativi, date, numeri e valori numerici precisi.
3.  Riscrivi definizioni con parole diverse ma mantenendo il significato tecnico identico.
4.  Riformula elenchi mantenendo ordine, struttura e significato. Parafrasa il testo di ogni punto.
5.  Dividi frasi lunghe se necessario per chiarezza, mantenendo coerenza.

Gestione Placeholders Immagine (POSIZIONAMENTO IMPORTANTE):
6.  Nel testo input troverai placeholders del tipo IMGP(NumPag)xI(IdxImg). Questi rappresentano immagini e sono già stati posizionati nel flusso del testo in base all'ordine degli elementi (testo/immagini) nel documento PDF originale.
7.  Il tuo compito è:
    a. Parafrasare il testo che si trova *attorno* a questi placeholder.
    b. MANTENERE il placeholder ESATTO (es. IMGP1xI1) e lasciarlo NELLA POSIZIONE RELATIVA in cui lo trovi rispetto al testo circostante. Non spostarlo altrove. Verrà sostituito dopo con il tag immagine corretto.
8.  NON inventare o aggiungere ALTRI placeholders IMGP...xI....

Gestione Tabelle, Valuta, Formattazione, Errori:
9.  Tabelle (FORMATTAZIONE MARKDOWN con Spaziatura):
    - Se identifichi una tabella, FAI DEL TUO MEGLIO PER RICOSTRUIRLA in MARKDOWN STANDARD: usa il carattere pipe (|) per separare le colonne e una riga separatrice composta da trattini e pipe (es. |---|---|...). Preserva tutti i dati.
    - IMPORTANTE: Assicurati che ci sia SEMPRE una riga vuota (un "a capo" extra) sia PRIMA della prima riga della tabella (quella con gli header) sia DOPO l'ultima riga della tabella, per separarla correttamente dal testo circostante.
    - Esempio di output corretto nel Markdown (nota le righe vuote immaginarie prima e dopo):

      Testo che precede la tabella.

      | Header1   | Header2   |
      |-----------|-----------|
      | Riga1Col1 | Riga1Col2 |

      Testo che segue la tabella.

    - Se la tabella originale è troppo ambigua o complessa, usa liste o testo descrittivo. Se l'input ha già una tabella formattata in Markdown, preservala assicurandoti comunque che abbia le righe vuote attorno.
10. Formule/Equazioni/Notazioni Matematiche/Scientifiche: Vedi punto 14.
11. Formatta l'output finale in MARKDOWN: Usa due cancelletti seguiti da spazio per i titoli H2 (## H2), tre cancelletti seguiti da spazio per i titoli H3 (### H3), due asterischi attorno a una parola per il grassetto (**grassetto**), un asterisco attorno a una parola per il corsivo (*corsivo*), liste puntate con un asterisco iniziale seguito da spazio (* punto elenco), o liste numerate con numero punto spazio (1. punto elenco). Mantieni la gerarchia e l'indentazione originali come nel testo di input.
12. Note a piè di pagina: Parafrasa il contenuto mantenendo il riferimento (esempio: [1] Testo parafrasato.).
13. Indica testo illeggibile scrivendo esattamente [TESTO ILLEGGIBILE].
14. Gestione FORMULE e NOTAZIONI SCIENTIFICHE (SOLO LaTeX, NIENTE Backticks!):
    - NON modificare il contenuto interno delle formule.
    - Usa ESCLUSIVAMENTE i delimitatori LaTeX standard: $ ... $ per formule inline, $$ ... $$ per formule display.
    - IMPORTANTISSIMO: NON USARE MAI i backtick (`) per la matematica. I backtick sono SOLO per il codice (es. `var=5`). Usa sempre e solo $formula$ o $$formula$$.
    - Parafrasa solo il testo attorno alle formule, lasciando la formula e i suoi delimitatori $/$$ intatti.
    - Non inventare formule. Esempi corretti nel Markdown finale: La relazione $p^M = a - bx^M(T)$ è importante. La famosa equazione è $$E=mc^2$$.
15. Gestione Specifica Valuta Dollaro (IMPORTANTE): Se devi usare il simbolo dollaro USA per valuta (esempio: 120 dollari), USA SEMPRE una barra rovesciata seguita dal simbolo dollaro (\$) nel Markdown. Esempio: Il prezzo è 120\$.
16. Restrizioni Aggiuntive sull'Output:
    - NON generare MAI linee orizzontali di separazione (usando ---, ***, o ___).
    - NON fare MAI riferimento ai numeri di pagina o alla struttura del documento originale nel testo parafrasato.
17. Output Finale: Produci UNICAMENTE il testo Markdown risultante dalla parafrasi. Deve contenere: placeholders IMGP... (posizionati correttamente nel flusso), tabelle Markdown corrette e spaziate con righe vuote prima e dopo, formule LaTeX corrette ($ o $$, NO backticks), valuta \$ corretta. NON includere: titoli aggiuntivi, header, footer, commenti, linee orizzontali, o altro testo non richiesto.
18. Scrivi nella lingua del testo originale.
"""

# --- 5. Funzioni di Utilità ---

def extract_text_images_annotations(pdf_content_bytes, temp_image_folder):
    """Estrae testo/immagini PDF usando l'analisi del layout (get_text("dict"))."""
    page_texts = []; image_references = {}; doc = None
    print("Avvio estrazione testo/immagini PDF con analisi layout (get_text dict)...")
    try:
        if not os.path.exists(temp_image_folder): os.makedirs(temp_image_folder); print(f"Cartella temporanea creata: {temp_image_folder}")
        doc = fitz.open(stream=pdf_content_bytes, filetype="pdf"); n_pages = doc.page_count; print(f"PDF caricato. Pagine: {n_pages}"); image_counter_total = 0
        for i in range(n_pages):
            page_num_actual = i + 1; page = doc.load_page(i)
            current_page_content_parts = [] ; img_count_on_page = 0
            try:
                page_dict = page.get_text("dict", sort=True) # Rimosso flags
                for block in page_dict["blocks"]:
                    if block["type"] == 0: # Blocco di testo
                        block_text = "";
                        for line in block["lines"]:
                            line_text = "";
                            for span in line["spans"]: line_text += span["text"]
                            block_text += line_text + " "
                        cleaned_block_text = unicodedata.normalize('NFC', block_text.strip());
                        if cleaned_block_text: current_page_content_parts.append(cleaned_block_text)
                    elif block["type"] == 1: # Blocco immagine
                        try:
                            img_count_on_page += 1; image_counter_total += 1; placeholder = f"IMGP{page_num_actual}xI{img_count_on_page}"; img_filename = f"{placeholder}.png"
                            local_image_path = os.path.join(temp_image_folder, img_filename); image_bytes = block["image"]
                            try:
                                pil_image = Image.open(io.BytesIO(image_bytes));
                                if pil_image.mode == 'P': pil_image = pil_image.convert('RGBA')
                                if pil_image.mode != 'RGB': pil_image = pil_image.convert('RGB')
                                pil_image.save(local_image_path, "PNG"); image_references[placeholder] = {'path': local_image_path, 'filename': img_filename}; current_page_content_parts.append(placeholder)
                            except Exception as save_e: print(f"      ! Warn: Salvataggio/Processo PIL img dict Pag {page_num_actual}, Blocco {block['number']}: {save_e}. IGNORATA.")
                        except Exception as img_extract_e: print(f"      ! Warn: Errore estrazione/dati immagine Blocco {block.get('number', 'N/A')} Pag {page_num_actual}: {img_extract_e}")
                page_final_text = "\n\n".join(current_page_content_parts); page_texts.append(page_final_text)
            except Exception as dict_err:
                print(f"Errore elaborazione blocchi (get_text dict) Pag {page_num_actual}: {dict_err}. Fallback a testo semplice.");
                try:
                    fallback_text = unicodedata.normalize('NFC', page.get_text("text", sort=True) or "")
                    page_texts.append(fallback_text.strip() + "\n\n[ERRORE_ESTRAZIONE_LAYOUT_PAGINA - Immagini Mancanti]\n\n"); print(f"      Fallback OK Pag {page_num_actual}. ATTENZIONE: Immagini perse.")
                except Exception as fallback_err: print(f"Errore anche nel fallback get_text Pag {page_num_actual}: {fallback_err}"); page_texts.append(f"\n\n[ERRORE_ESTRAZIONE_TOTALE_PAGINA_{page_num_actual}]\n\n")
        print(f"Estrazione PDF con analisi layout completata. Img temp referenziate: {len(image_references)}.")
        return page_texts, image_references
    except fitz.fitz.FileDataError as pdf_err: print(f"Errore Fatale Apertura PDF: {pdf_err}"); traceback.print_exc(); return None, None
    except Exception as e: print(f"Errore Fatale Estrazione PDF: {e}"); traceback.print_exc(); return None, None
    finally:
        if doc: doc.close(); print("Documento PyMuPDF chiuso.")


def paraphrase_text_chunk(text_chunk, instructions, model, chunk_header):
    """Invia chunk a Gemini per parafrasi."""
    if not text_chunk.strip(): print(f"Chunk '{chunk_header}' vuoto, saltato."); return ""
    prompt = f"""{instructions}

--- INIZIO TESTO DA PARAFRASARE ({chunk_header}) ---
{text_chunk}
--- FINE TESTO DA PARAFRASARE ({chunk_header}) ---

Fornisci SOLO il testo parafrasato in MARKDOWN per '{chunk_header}', seguendo TUTTE le istruzioni aggiornate (mantieni `IMGP...` NELLA LORO POSIZIONE ESATTA, formatta tabelle **CON RIGHE VUOTE PRIMA E DOPO**, formule LaTeX `$..$` o `$$..$$` SENZA USARE MAI backtick \` per la matematica, dollari valuta come `\$`, NON usare linee orizzontali ---, NON riferirti a numeri pagina originali). **NON INCLUDERE commenti, header, footer o altro testo al di fuori del Markdown risultante.**"""
    max_retries = 3; base_delay = 5
    for attempt in range(max_retries):
        print(f"      Tentativo {attempt + 1}/{max_retries} invio chunk '{chunk_header}' a Gemini ({model_name_in_use})...")
        paraphrased_text = None; final_text = ""; finish_reason_name = "UNKNOWN"; safety_ratings_str = "N/A"; candidate = None
        try:
            response = model.generate_content(prompt); block_reason=None
            if response.prompt_feedback: safety_ratings_str=str(response.prompt_feedback.safety_ratings) if response.prompt_feedback.safety_ratings else"N/A";block_reason=str(response.prompt_feedback.block_reason) if response.prompt_feedback.block_reason else None
            if block_reason: print(f"ATTENZIONE: Blocco PROMPT '{chunk_header}'(T{attempt+1}):{block_reason}. Sicurezza:{safety_ratings_str}")
            if not response.candidates: print(f"ATTENZIONE: No candidati '{chunk_header}'(T{attempt+1}). Sicurezza Prompt:{safety_ratings_str}")
            else:
                candidate=response.candidates[0]
                if candidate.safety_ratings:resp_safety_str=str(candidate.safety_ratings);safety_ratings_str=f"{safety_ratings_str}|R:{resp_safety_str}" if safety_ratings_str!="N/A" else f"R:{resp_safety_str}"
                finish_reason_name=candidate.finish_reason.name if candidate.finish_reason else "UNKNOWN"
                if finish_reason_name=="STOP":paraphrased_text=response.text
                elif finish_reason_name=="MAX_TOKENS":paraphrased_text=response.text;print(f"ATTENZIONE:MAX_TOKENS '{chunk_header}'(T{attempt+1}). Output potrebbe essere incompleto.")
                elif finish_reason_name=="SAFETY":print(f"ATTENZIONE:Blocco SAFETY RISPOSTA '{chunk_header}'(T{attempt+1}).Sicurezza:{safety_ratings_str}");paraphrased_text=None;
                elif finish_reason_name in["RECITATION","OTHER"]:print(f"ATTENZIONE:Terminazione anomala '{chunk_header}'(T{attempt+1}):{finish_reason_name}.Sicurezza:{safety_ratings_str}");paraphrased_text=None;
                else:print(f"ATTENZIONE:Terminazione non gestita '{chunk_header}'(T{attempt+1}):{finish_reason_name}.Sicurezza:{safety_ratings_str}");paraphrased_text=None;
            if paraphrased_text is not None:
                cleaned_text=paraphrased_text.strip();lines=cleaned_text.splitlines();
                lines_filtered=[l for l in lines if l.strip() and not l.strip().startswith("--- INIZIO") and not l.strip().startswith("--- FINE") and not l.strip() == f"--- {chunk_header} ---" and not re.match(r"^\s*[`]*(markdown)?\s*$", l, re.IGNORECASE) and not re.match(r"^\s*([-*_]\s*){3,}\s*$", l)]
                if lines_filtered and lines_filtered[-1].strip()=="```":lines_filtered.pop();
                if lines_filtered and lines_filtered[0].strip()=="```":lines_filtered.pop(0);
                final_text="\n".join(lines_filtered).strip();
                if not final_text and finish_reason_name=="STOP":print(f"      Info/Warn:Risposta '{chunk_header}' valida(STOP)ma vuota post-pulizia.")
            should_retry=(finish_reason_name in["SAFETY","RECITATION","OTHER"]or not candidate or block_reason)
            if(final_text or(paraphrased_text is not None and finish_reason_name=="MAX_TOKENS")or(paraphrased_text==""and finish_reason_name=="STOP"))and not should_retry:
                status_msg=f"(Nota:Troncato? {finish_reason_name})"if finish_reason_name!='STOP'else"(Completato)";print(f"      Chunk '{chunk_header}' parafrasato(T{attempt+1}).{status_msg}");return final_text
            if attempt<max_retries-1:
                if should_retry: delay=base_delay*(2**attempt);print(f"        -> Retry ({block_reason or finish_reason_name or 'No candidati'}).Attesa {delay}s...");time.sleep(delay);continue
                else: print(f"ERRORE INTERNO? T{attempt+1} '{chunk_header}' non riuscito ma non retryable? Motivo:{finish_reason_name},BloccoP:{block_reason}. Tentativo Retry...");delay=base_delay*(2**attempt);time.sleep(delay);continue
            else: # Fallimento definitivo
                err_msg = f"ERRORE FINALE API/GENERAZIONE per {chunk_header} dopo {max_retries} tentativi.\n"; err_msg += f"- Ultimo Motivo: {finish_reason_name}\n"; err_msg += f"- Blocco Prompt: {block_reason}\n"; err_msg += f"- Dettagli Sicurezza: {safety_ratings_str}\n"
                if paraphrased_text is not None: err_msg += f"- Ultimo Output (potrebbe essere parziale):\n{paraphrased_text.strip()}\n"
                else: err_msg += "- Nessun output valido generato nell'ultimo tentativo.\n"
                print(f"ERRORE FINALE (dopo retries) per {chunk_header}.")
                return f"\n\n```error\n[ERRORE DI ELABORAZIONE: {chunk_header}]\n\n{err_msg}\n```\n\n"
        except Exception as e:
            print(f"ERRORE CATTURATO API/ELAB '{chunk_header}'(T{attempt+1}):{e}");
            if not isinstance(e,(genai.types.StopCandidateException,genai.types.BlockedPromptException,genai.types.generation_types.BrokenResponseError,genai.types.generation_types.IncompleteIterationError)):traceback.print_exc()
            if attempt==max_retries-1:
                error_final_msg = f"[ERRORE DI ELABORAZIONE (Eccezione): {chunk_header}]\n\nEccezione: {e}\n"; print(f"ERRORE ECCEZIONE FINALE (dopo retries) per '{chunk_header}'.")
                return f"\n\n```error\n{error_final_msg}\n```\n\n"
            else: delay=base_delay*(2**attempt);print(f"        -> Errore, ritento tra {delay}s...");time.sleep(delay)
    print(f"ERRORE IMPREVISTO Loop '{chunk_header}'.")
    return f"\n\n```error\n[ERRORE LOGICA SCRIPT: {chunk_header}]\nFalliti {max_retries} tentativi in modo imprevisto.\n```\n\n"


def filter_and_save_valid_images(image_references, temp_image_folder, final_drive_image_folder, black_threshold=10, white_threshold=245):
    """Analizza img temp DAL PDF, copia valide in Drive, restituisce dict img valide e set placeholder rimossi."""
    # ... (Codice Invariato da v4.13.1) ...
    print(f"\n--- Filtro Immagini Vuote/Nere (dal PDF) e Copia Valide ---"); print(f"Analisi da: {temp_image_folder}"); print(f"Copia valide in: {final_drive_image_folder}"); print(f"Soglie: Nero < {black_threshold}, Bianco > {white_threshold}")
    valid_image_refs = {}; removed_placeholders = set(); copied_count = 0; error_count = 0; blank_count = 0
    if not image_references: print("Nessuna immagine iniziale dal PDF da analizzare."); return {}, set()
    try: os.makedirs(final_drive_image_folder, exist_ok=True)
    except OSError as e: print(f"!!! ERRORE CRITICO: Creazione cartella img dest '{final_drive_image_folder}': {e}"); return {}, set(image_references.keys())
    for placeholder, img_data in image_references.items():
        temp_path = img_data.get('path'); filename = img_data.get('filename')
        if not temp_path or not filename or not os.path.isfile(temp_path): print(f"      ! Warn: Ref/file non valido per {placeholder} (path: {temp_path}). Rimossa."); removed_placeholders.add(placeholder); error_count += 1; continue
        is_blank = False
        try:
            with Image.open(temp_path) as img: grayscale_img = img.convert('L'); min_val, max_val = grayscale_img.getextrema(); is_blank = max_val < black_threshold or min_val > white_threshold
        except UnidentifiedImageError: print(f"      ! Warn: Impossibile aprire/identificare {filename}. Rimossa."); removed_placeholders.add(placeholder); error_count += 1; continue
        except Exception as e: print(f"      ! Warn: Errore analisi immagine {filename}: {e}. Rimossa."); removed_placeholders.add(placeholder); error_count += 1; continue
        if is_blank: print(f"      - Rilevata immagine vuota/quasi vuota (PDF): {filename}. Rimossa."); removed_placeholders.add(placeholder); blank_count += 1
        else:
            final_path = os.path.join(final_drive_image_folder, filename)
            try: shutil.copy2(temp_path, final_path); valid_image_refs[placeholder] = {'filename': filename}; copied_count += 1
            except Exception as e: print(f"      ! ERRORE copia img PDF {filename} in Drive: {e}. Rimossa."); removed_placeholders.add(placeholder); error_count += 1
    print(f"Filtro immagini PDF completato. Valide copiate: {copied_count}. Vuote rimosse: {blank_count}. Errori: {error_count}.")
    return valid_image_refs, removed_placeholders


def mount_drive():
    """Monta Google Drive."""
    # ... (Codice Invariato da v4.13.1) ...
    try: drive.mount('/content/drive', force_remount=True); print("Google Drive montato con successo."); return True
    except Exception as e: print(f"Errore Critico durante il montaggio di Google Drive: {e}"); return False


# <<< NUOVA FUNZIONE v4.13.2: Post-Processing per spaziatura tabelle >>>
def add_space_around_tables(markdown_text):
    """
    Analizza il testo Markdown e aggiunge una riga vuota prima e dopo
    i blocchi di tabelle Markdown, se non già presenti.
    """
    print("  Applicazione post-processing per spaziatura tabelle...")
    lines = markdown_text.split('\n')
    new_lines = []
    num_lines = len(lines)
    added_before = 0
    added_after = 0

    for i, line in enumerate(lines):
        # Strip per ignorare spazi bianchi nella riga ai fini del controllo '|'
        stripped_line = line.strip()
        current_line_is_table = stripped_line.startswith('|') and stripped_line.endswith('|')

        # Controllo riga precedente
        prev_line_is_blank = (i == 0) or (lines[i-1].strip() == "")
        prev_line_is_table = (i > 0) and lines[i-1].strip().startswith('|') and lines[i-1].strip().endswith('|')

        # Aggiungi riga vuota PRIMA se inizia una tabella e la riga prima non è vuota/tabella
        if current_line_is_table and not prev_line_is_table and not prev_line_is_blank and i > 0:
            new_lines.append("")
            added_before += 1

        # Aggiungi la riga corrente
        new_lines.append(line)

        # Controllo riga successiva
        next_line_is_blank = (i == num_lines - 1) or (lines[i+1].strip() == "")
        next_line_is_table = (i < num_lines - 1) and lines[i+1].strip().startswith('|') and lines[i+1].strip().endswith('|')

        # Aggiungi riga vuota DOPO se finisce una tabella e la riga dopo non è vuota/tabella
        if current_line_is_table and not next_line_is_table and not next_line_is_blank and i < num_lines - 1:
            new_lines.append("")
            added_after += 1

    if added_before > 0 or added_after > 0:
        print(f"  Aggiunte {added_before} righe vuote prima e {added_after} dopo le tabelle.")
        return "\n".join(new_lines)
    else:
        print("  Nessuna riga vuota aggiuntiva necessaria per le tabelle.")
        return markdown_text # Restituisce il testo originale se non sono state fatte modifiche

# <<< FINE NUOVA FUNZIONE v4.13.2 >>>


# --- 6. Funzione Principale di Esecuzione ---
def run_full_process():
    """Flusso: setup, Drive, PDF upload, ESTRAZIONE LAYOUT, parafrasi, PULIZIA FORMULE, **PULIZIA SPAZIO TABELLE**, filtro/copia img PDF, COPIA LOGO FISSO, post-proc MD img, aggiunta logo, salvataggio MD."""
    print("=============================================")
    print(f"--- Avvio Processo Parafrasi PDF v4.13.2 - FIX TABLE SPACE POSTPROC + LAYOUT IMG POS ({os.path.basename(PERMANENT_HEADER_LOGO_SOURCE_PATH)}) ---")
    print("=============================================")
    CHUNK_SIZE_PAGES = 20
    TEMP_IMAGE_FOLDER = "/content/temp_pdf_parafrasi_images_v4_13" # Usa stessa temp folder
    DRIVE_BASE_OUTPUT_FOLDER = "/content/drive/MyDrive/Appunti TopNotes/markdown"
    print(f"Percorso base di output impostato su: {DRIVE_BASE_OUTPUT_FOLDER}")
    print(f"Percorso Logo Header FISSO su Drive: {PERMANENT_HEADER_LOGO_SOURCE_PATH}")

    # Pulizia/Creazione cartella temporanea
    if os.path.exists(TEMP_IMAGE_FOLDER): print(f"Pulizia cartella temporanea precedente: {TEMP_IMAGE_FOLDER}");
    try: shutil.rmtree(TEMP_IMAGE_FOLDER);
    except Exception as e: print(f"Warn: Errore rimozione temp: {e}")
    try: os.makedirs(TEMP_IMAGE_FOLDER)
    except Exception as e: print(f"Errore Critico: Impossibile creare cartella temporanea '{TEMP_IMAGE_FOLDER}': {e}"); return

    # --- 1. Montaggio Google Drive ---
    print("\n--- 1. Montaggio Google Drive ---")
    if not mount_drive(): return

    # --- 2. Configurazione Output su Drive (Preliminare) ---
    timestamp = time.strftime("%Y%m%d_%H%M%S")
    uploaded_file_name_base = "PDF_Parafrasato"; drive_output_subfolder_base = DRIVE_BASE_OUTPUT_FOLDER
    uploaded_file_name = None; output_md_path = None; drive_output_subfolder = None
    drive_image_folder_relative = "Images"; drive_image_folder_absolute = None
    header_image_filename_dest = "logo.jpg"; header_image_copy_path_dest = None
    header_image_copied_successfully = False

    # --- 3. Caricamento File PDF ---
    print("\n--- 3. Caricamento File PDF ---"); print("Seleziona e carica il file PDF...")
    uploaded_pdf_content = None
    try:
        uploaded = files.upload()
        if not uploaded: print("Nessun file PDF caricato. Processo interrotto."); return
        uploaded_file_name = list(uploaded.keys())[0]; uploaded_pdf_content = uploaded[uploaded_file_name]
        print(f"File PDF '{uploaded_file_name}' caricato ({len(uploaded_pdf_content)} bytes).")
        sanitized_file_name = re.sub(r'[\\/*?:"<>|]', "_", os.path.splitext(uploaded_file_name)[0])
        uploaded_file_name_base = f"{sanitized_file_name}_{timestamp}"
        drive_output_subfolder = os.path.join(drive_output_subfolder_base, uploaded_file_name_base)
        output_md_filename = f"{uploaded_file_name_base}_parafrasato.md"
        output_md_path = os.path.join(drive_output_subfolder, output_md_filename)
        drive_image_folder_absolute = os.path.join(drive_output_subfolder, drive_image_folder_relative)
        header_image_copy_path_dest = os.path.join(drive_image_folder_absolute, header_image_filename_dest)
        print(f"Cartella Output: {drive_output_subfolder}"); print(f"File Markdown: {output_md_path}"); print(f"Cartella Immagini: {drive_image_folder_absolute}"); print(f"Logo Destinazione: {header_image_copy_path_dest}")
        os.makedirs(drive_image_folder_absolute, exist_ok=True); print(f"Cartelle Drive create/verificate.")
    except OSError as e: print(f"ERRORE CRITICO Creazione Cartelle Output Drive: {e}"); return
    except Exception as e: print(f"ERRORE CRITICO Upload PDF / Setup Percorsi: {e}"); traceback.print_exc(); return

    # --- 4. Estrazione Contenuto PDF con Analisi Layout ---
    print("\n--- 4. Estrazione Contenuto PDF (con Analisi Layout) ---")
    page_texts, initial_image_references = extract_text_images_annotations(uploaded_pdf_content, TEMP_IMAGE_FOLDER)
    if page_texts is None or initial_image_references is None : print("ERRORE CRITICO Estrazione PDF. Processo interrotto."); return
    print(f"Estrazione PDF con Analisi Layout completata. Pagine: {len(page_texts)}. Img PDF iniziali: {len(initial_image_references)}")

    # --- 5. Parafrasi con Gemini ---
    print(f"\n--- 5. Parafrasi Testo con Gemini ({model_name_in_use}) ---")
    full_paraphrased_markdown_with_placeholders = ""; total_pages = len(page_texts)
    total_chunks = math.ceil(total_pages / CHUNK_SIZE_PAGES) if total_pages > 0 else 0
    print(f"Divisione in {total_chunks} chunk da {CHUNK_SIZE_PAGES} pagine (circa).")
    md_save_success = False; chunk_processing_errors = 0
    if total_chunks == 0 and not initial_image_references: print("PDF vuoto. MD conterrà solo header (logo)."); full_paraphrased_markdown_with_placeholders = ""
    elif total_chunks == 0 and initial_image_references: print("PDF solo immagini. MD conterrà header + placeholder img PDF."); full_paraphrased_markdown_with_placeholders = "\n\n".join(initial_image_references.keys()) + "\n\n"
    else:
        for i in range(0, total_pages, CHUNK_SIZE_PAGES):
            chunk_num=(i//CHUNK_SIZE_PAGES)+1; chunk_header = f"CHUNK_{chunk_num}"; print(f"\n  Processando {chunk_header}/{total_chunks}...")
            text_chunk_to_process = "\n\n".join(page_texts[i:min(i + CHUNK_SIZE_PAGES, total_pages)])
            paraphrased_chunk_md = paraphrase_text_chunk(text_chunk_to_process, PARAPHRASING_INSTRUCTIONS, model, chunk_header)
            if "```error" in paraphrased_chunk_md: chunk_processing_errors+=1; print(f"!!! Errore API {chunk_header}. Inserito nel MD.");
            full_paraphrased_markdown_with_placeholders += paraphrased_chunk_md
            if not "```error" in paraphrased_chunk_md and paraphrased_chunk_md.strip(): full_paraphrased_markdown_with_placeholders += "\n\n"
            if chunk_num < total_chunks: pause_duration = 1.5; print(f"      ...Pausa {pause_duration}s..."); time.sleep(pause_duration)
        print("\n--- Parafrasi chunk completata ---")
        if chunk_processing_errors > 0: print(f"ATTENZIONE: {chunk_processing_errors} errori API durante parafrasi chunk (vedere blocchi 'error' nel MD).")
        else: print("Parafrasi chunk completata senza errori API apparenti.")

    # --- 6. Pulizia Formule LaTeX ---
    print("\n--- 6. Pulizia Formule LaTeX (Rimozione Backtick Errati) ---")
    cleaned_markdown_formulas = full_paraphrased_markdown_with_placeholders; count_backticks_removed = 0
    try:
        cleaned_markdown_formulas, count1 = re.subn(r'`(\$[^\$]+\$)`', r'\1', cleaned_markdown_formulas)
        cleaned_markdown_formulas, count2 = re.subn(r'`(\$\$.*?\$\$)`', r'\1', cleaned_markdown_formulas, flags=re.DOTALL)
        count_backticks_removed = count1 + count2
        if count_backticks_removed > 0: print(f"  Rimossi {count_backticks_removed} backtick singoli adiacenti a formule $/$$..."); full_paraphrased_markdown_with_placeholders = cleaned_markdown_formulas
        else: print("  Nessun backtick singolo direttamente adiacente a $/$$ trovato.")
    except Exception as regex_e: print(f"  WARN: Errore pulizia regex formule: {regex_e}.")

    # <<< NUOVO STEP v4.13.2: Pulizia Spaziatura Tabelle >>>
    print("\n--- 7. Pulizia Spaziatura Tabelle Markdown ---")
    try:
        full_paraphrased_markdown_with_placeholders = add_space_around_tables(full_paraphrased_markdown_with_placeholders)
    except Exception as table_space_e:
        print(f"  WARN: Errore durante l'aggiunta di spazio attorno alle tabelle: {table_space_e}. Si procede senza questa pulizia.")
    # <<< FINE NUOVO STEP v4.13.2 >>>

    # --- 8. Filtro Immagini PDF e Copia Logo Fisso --- (Era Step 7)
    print("\n--- 8. Filtro Immagini PDF e Copia Logo Fisso su Drive ---")
    valid_image_refs, removed_placeholders = filter_and_save_valid_images(initial_image_references, TEMP_IMAGE_FOLDER, drive_image_folder_absolute)
    print(f"Placeholders img PDF validi: {len(valid_image_refs)}"); print(f"Placeholders img PDF rimossi: {len(removed_placeholders)}")
    print(f"  Tentativo copia Logo Header da: {PERMANENT_HEADER_LOGO_SOURCE_PATH}")
    if not PERMANENT_HEADER_LOGO_SOURCE_PATH: print("  WARN: Percorso logo fisso non configurato."); header_image_copied_successfully = False
    elif not os.path.exists(PERMANENT_HEADER_LOGO_SOURCE_PATH): print(f"  !!! ERRORE CRITICO: Logo SORGENTE non trovato: {PERMANENT_HEADER_LOGO_SOURCE_PATH}"); header_image_copied_successfully = False
    elif not header_image_copy_path_dest: print("  !!! ERRORE INTERNO: Percorso destinazione logo non definito."); header_image_copied_successfully = False
    else:
        try: shutil.copy2(PERMANENT_HEADER_LOGO_SOURCE_PATH, header_image_copy_path_dest); header_image_copied_successfully = True; print(f"  Logo Header copiato in: {header_image_copy_path_dest}")
        except Exception as e: print(f"  !!! ERRORE copia Logo Header: {e}"); header_image_copied_successfully = False

    # --- 9. Post-Elaborazione Markdown Immagini PDF --- (Era Step 8)
    print("\n--- 9. Post-Elaborazione Markdown Immagini PDF (Sostituzione Placeholder) ---")
    # Usa il markdown già pulito per formule E tabelle
    final_markdown_content = full_paraphrased_markdown_with_placeholders
    placeholders_replaced_count = 0; placeholders_removed_count = 0
    if valid_image_refs:
        print("  Sostituzione placeholder img PDF validi..."); sorted_placeholders = sorted(valid_image_refs.keys(), key=len, reverse=True)
        for placeholder in sorted_placeholders:
            img_filename = valid_image_refs[placeholder]['filename']; relative_img_path = f"./{drive_image_folder_relative}/{img_filename}"; markdown_tag = f"![{placeholder}]({relative_img_path})"
            final_markdown_content, count = re.subn(re.escape(placeholder), markdown_tag, final_markdown_content, flags=re.IGNORECASE)
            if count > 0: placeholders_replaced_count += count
        print(f"  Sostituiti {placeholders_replaced_count} placeholder img PDF validi.")
    else: print("  Nessun placeholder img PDF valido da sostituire.")
    if removed_placeholders:
        print("  Rimozione placeholder img PDF non validi..."); placeholders_to_remove_sorted = sorted(list(removed_placeholders), key=len, reverse=True)
        for placeholder in placeholders_to_remove_sorted:
             pattern_placeholder_only = r'\s*' + re.escape(placeholder) + r'\s*\n?'
             final_markdown_content, count = re.subn(pattern_placeholder_only, '', final_markdown_content, flags=re.IGNORECASE | re.MULTILINE)
             if count > 0: placeholders_removed_count += count
        print(f"  Rimossi {placeholders_removed_count} placeholder img PDF non validi.")
    else: print("  Nessun placeholder img PDF non valido da rimuovere.")

    # --- 10. Composizione Finale (Solo Logo Header) --- (Era Step 9)
    print("  Composizione Header finale (Solo Logo Fisso)...")
    header_content = ""
    if header_image_copied_successfully and header_image_copy_path_dest:
        header_image_relative_path = f"./{drive_image_folder_relative}/{header_image_filename_dest}"; header_content += f"![Logo Header]({header_image_relative_path})\n\n"; print(f"  Aggiunto tag per Logo Header: {header_image_relative_path}")
    else: print("  Logo Header non copiato o errore, non aggiunto al MD.")
    final_markdown_output = header_content + final_markdown_content.strip()
    print("\n--- Controllo Finale Output MD ---"); print(f"Lunghezza finale MD: {len(final_markdown_output)} caratteri."); print("------------------------------------\n")

    # --- 11. Salvataggio File Markdown Finale su Drive --- (Era Step 10)
    print("\n--- 11. Salvataggio File Markdown Finale su Drive ---")
    if output_md_path:
        try:
            os.makedirs(os.path.dirname(output_md_path), exist_ok=True)
            with open(output_md_path, 'w', encoding='utf-8') as f: f.write(final_markdown_output)
            print(f"File Markdown finale salvato con successo in: {output_md_path}"); md_save_success = True
        except Exception as e: print(f"!!! ERRORE CRITICO salvataggio MD finale: {e}"); traceback.print_exc(); md_save_success = False
    else: print("Errore: Percorso MD output non definito."); md_save_success = False

    # --- 12. Pulizia Cartella Temporanea --- (Era Step 11)
    print("\n--- 12. Pulizia ---")
    if os.path.exists(TEMP_IMAGE_FOLDER):
        print(f"Rimozione cartella temporanea: {TEMP_IMAGE_FOLDER}");
        try: shutil.rmtree(TEMP_IMAGE_FOLDER); print("Cartella temporanea rimossa.")
        except Exception as e: print(f"Warn: Errore rimozione temp: {e}")
    else: print("Cartella temporanea non trovata.")

    # --- Riepilogo Finale ---
    print("\n=============================================")
    # ... (Logica Riepilogo Invariata) ...
    final_outcome = "ERRORE";
    if md_save_success: final_outcome = "COMPLETATO CON SUCCESSO"
    else: final_outcome = "TERMINATO CON ERRORI (Salvataggio MD Fallito/Non Tentato)"
    if chunk_processing_errors > 0 and md_save_success: final_outcome = "COMPLETATO (ma con errori API nei chunk - controllare MD)"
    if not header_image_copied_successfully and os.path.exists(PERMANENT_HEADER_LOGO_SOURCE_PATH) and md_save_success: final_outcome = "COMPLETATO (ma con ERRORE COPIA LOGO FISSO)"
    elif not header_image_copied_successfully and not os.path.exists(PERMANENT_HEADER_LOGO_SOURCE_PATH) and md_save_success: final_outcome = "COMPLETATO (ma ATTENZIONE: LOGO FISSO SORGENTE non trovato)"

    print(f"--- PROCESSO {final_outcome} ---")
    if output_md_path and md_save_success and os.path.isfile(output_md_path): print(f"File Markdown salvato su Drive in: {output_md_path}")
    elif output_md_path and md_save_success: print(f"ATTENZIONE: Salvataggio MD reportato OK, ma file non trovato in: {output_md_path}")
    elif output_md_path: print(f"Errore: File Markdown NON salvato. Doveva essere in: {output_md_path}")
    else: print("Errore: Percorso file Markdown non definito.")

    final_pdf_image_count = -1; logo_found_in_dest = False
    if drive_image_folder_absolute and os.path.isdir(drive_image_folder_absolute):
        try:
            all_files = [f for f in os.listdir(drive_image_folder_absolute) if os.path.isfile(os.path.join(drive_image_folder_absolute, f))]
            pdf_images = [f for f in all_files if f.lower().startswith('imgp') and f.lower().endswith('.png')]
            final_pdf_image_count = len(pdf_images); logo_found_in_dest = header_image_filename_dest in all_files
            print(f"Cartella Immagini: {drive_image_folder_absolute} ({final_pdf_image_count} img PDF, Logo trovato: {logo_found_in_dest})")
            if not header_image_copied_successfully and not logo_found_in_dest: print(f"  (Logo header non copiato/trovato - controllare sorgente: {PERMANENT_HEADER_LOGO_SOURCE_PATH})")
        except Exception as list_e: print(f"Cartella Immagini creata ({drive_image_folder_absolute}), ma errore nel listare i file: {list_e}")
    elif len(initial_image_references) > 0 : print(f"Errore: C'erano immagini PDF ma cartella immagini Drive ({drive_image_folder_absolute}) non trovata.")
    else: print("Nessuna immagine PDF è stata processata.")

    if not md_save_success or chunk_processing_errors > 0 or not header_image_copied_successfully:
        print("\nATTENZIONE: Il processo è terminato con errori o avvisi.")
        if not header_image_copied_successfully and not os.path.exists(PERMANENT_HEADER_LOGO_SOURCE_PATH): print(f"  >> CONTROLLA CHE IL FILE LOGO ESISTA IN: {PERMANENT_HEADER_LOGO_SOURCE_PATH} <<")
        elif not header_image_copied_successfully: print("  >> CONTROLLA I LOG PER ERRORI DURANTE LA COPIA DEL LOGO <<")
        if chunk_processing_errors > 0: print("  >> CONTROLLA I BLOCCHI 'error' NEL FILE MARKDOWN FINALE <<")
        if drive_output_subfolder and os.path.exists(drive_output_subfolder): print(f"Controllare output parziale in: {drive_output_subfolder}")
        elif not drive_output_subfolder: print("Cartella output non creata.")
    print("=============================================")


# --- 13. Esecuzione --- (Era Step 12)
run_full_process()

API Key Gemini caricata con successo.
Modello richiesto: gemini-2.5-flash-preview-04-17
Configurazione generazione: {'temperature': 0.6, 'top_p': 0.95, 'top_k': 64, 'max_output_tokens': 60000, 'response_mime_type': 'text/plain'}
Tentativo inizializzazione modello: gemini-2.5-flash-preview-04-17
Modello 'gemini-2.5-flash-preview-04-17' inizializzato con successo.
--- Avvio Processo Parafrasi PDF v4.13.2 - FIX TABLE SPACE POSTPROC + LAYOUT IMG POS (logo.jpg) ---
Percorso base di output impostato su: /content/drive/MyDrive/Appunti TopNotes/markdown
Percorso Logo Header FISSO su Drive: /content/drive/MyDrive/Appunti TopNotes/logo/logo.jpg
Pulizia cartella temporanea precedente: /content/temp_pdf_parafrasi_images_v4_13

--- 1. Montaggio Google Drive ---
Mounted at /content/drive
Google Drive montato con successo.

--- 3. Caricamento File PDF ---
Seleziona e carica il file PDF...


In [ ]:
# -*- coding: utf-8 -*-

# VERSIONE v4.13.2-MD_TABLESPACE_POSTPROC_LAYOUT:
# - AGGIUNTA post-elaborazione per forzare righe vuote prima/dopo tabelle Markdown.
# - Mantiene: Analisi Layout Img Pos, Fix BBox tuple, Fix spazio tabelle (prompt), No Hr/Pg/Tl, Logo fisso, Modello originale, etc.
# Basato su v4.13.1

# --- 0. Installazione Librerie ---
!pip install -q google-generativeai pymupdf google-colab Pillow

# --- 1. Importazioni ---
import google.generativeai as genai
from google.colab import userdata, files, drive
import fitz  # PyMuPDF
from fitz import Rect
import io
import time
import pathlib
import textwrap
import os
import re
import sys
import traceback
import math
import unicodedata
from PIL import Image, UnidentifiedImageError
import shutil

# --- CONFIGURAZIONE UTENTE ---
PERMANENT_HEADER_LOGO_SOURCE_PATH = "/content/drive/MyDrive/Appunti TopNotes/logo/logo.jpg"
# !!! ASSICURATI CHE QUESTO FILE ESISTA NELLA POSIZIONE SPECIFICATA !!!
# ---------------------------

# --- 2. Configurazione API Key Gemini ---
# (Invariato)
try:
    GOOGLE_API_KEY = userdata.get('GEMINI_API_KEY')
    if not GOOGLE_API_KEY: raise ValueError("API Key non fornita o non trovata nei Secret.")
    genai.configure(api_key=GOOGLE_API_KEY)
    print("API Key Gemini caricata con successo.")
except (userdata.SecretNotFoundError, ValueError) as ve: print(f"Errore Critico: Chiave API 'GEMINI_API_KEY' non trovata. {ve}"); sys.exit(1)
except Exception as e: print(f"Errore Critico Configurazione API: {e}"); sys.exit(1)

# --- 3. Impostazioni Modello Gemini ---
# (Invariato)
MODEL_NAME_REQUESTED = 'gemini-2.5-flash-preview-04-17'
MODEL_FALLBACK_1 = 'gemini-1.5-pro-latest'
MODEL_FALLBACK_2 = 'gemini-1.0-pro'
generation_config = { "temperature": 0.6, "top_p": 0.95, "top_k": 64, "max_output_tokens": 60000, "response_mime_type": "text/plain" }
safety_settings = [{"category": f"HARM_CATEGORY_{cat}", "threshold": "BLOCK_MEDIUM_AND_ABOVE"} for cat in ["HARASSMENT", "HATE_SPEECH", "SEXUALLY_EXPLICIT", "DANGEROUS_CONTENT"]]
model = None; model_name_in_use = None; model_candidates = [MODEL_NAME_REQUESTED, MODEL_FALLBACK_1, MODEL_FALLBACK_2]
print(f"Modello richiesto: {MODEL_NAME_REQUESTED}"); print(f"Configurazione generazione: {generation_config}")
for model_candidate in model_candidates:
    print(f"Tentativo inizializzazione modello: {model_candidate}")
    try: model = genai.GenerativeModel(model_name=model_candidate, safety_settings=safety_settings, generation_config=generation_config); model_name_in_use = model_candidate; print(f"Modello '{model_name_in_use}' inizializzato con successo."); break
    except Exception as e: print(f"Errore inizializzazione '{model_candidate}': {e}")
if model is None: print("Errore Critico: Nessun modello Gemini inizializzato."); sys.exit(1)


# --- 4. Istruzioni Dettagliate per la Parafrasi ---
# (Invariato)
PARAPHRASING_INSTRUCTIONS = """
Elabora il testo fornito seguendo rigorosamente queste istruzioni. Genera l'output ESCLUSIVAMENTE in formato MARKDOWN.

Parafrasi del testo:
1.  Riscrivi il testo fornito usando sinonimi, riformulazioni e diverse costruzioni grammaticali, conservando INTEGRALMENTE il significato originale e tutte le informazioni chiave.
2.  Mantieni ASSOLUTAMENTE INVARIATI: nomi propri, acronimi, sigle, riferimenti normativi, date, numeri e valori numerici precisi.
3.  Riscrivi definizioni con parole diverse ma mantenendo il significato tecnico identico.
4.  Riformula elenchi mantenendo ordine, struttura e significato. Parafrasa il testo di ogni punto.
5.  Dividi frasi lunghe se necessario per chiarezza, mantenendo coerenza.

Gestione Placeholders Immagine (POSIZIONAMENTO IMPORTANTE):
6.  Nel testo input troverai placeholders del tipo IMGP(NumPag)xI(IdxImg). Questi rappresentano immagini e sono già stati posizionati nel flusso del testo in base all'ordine degli elementi (testo/immagini) nel documento PDF originale.
7.  Il tuo compito è:
    a. Parafrasare il testo che si trova *attorno* a questi placeholder.
    b. MANTENERE il placeholder ESATTO (es. IMGP1xI1) e lasciarlo NELLA POSIZIONE RELATIVA in cui lo trovi rispetto al testo circostante. Non spostarlo altrove. Verrà sostituito dopo con il tag immagine corretto.
8.  NON inventare o aggiungere ALTRI placeholders IMGP...xI....

Gestione Tabelle, Valuta, Formattazione, Errori:
9.  Tabelle (FORMATTAZIONE MARKDOWN con Spaziatura):
    - Se identifichi una tabella, FAI DEL TUO MEGLIO PER RICOSTRUIRLA in MARKDOWN STANDARD: usa il carattere pipe (|) per separare le colonne e una riga separatrice composta da trattini e pipe (es. |---|---|...). Preserva tutti i dati.
    - IMPORTANTE: Assicurati che ci sia SEMPRE una riga vuota (un "a capo" extra) sia PRIMA della prima riga della tabella (quella con gli header) sia DOPO l'ultima riga della tabella, per separarla correttamente dal testo circostante.
    - Esempio di output corretto nel Markdown (nota le righe vuote immaginarie prima e dopo):

      Testo che precede la tabella.

      | Header1   | Header2   |
      |-----------|-----------|
      | Riga1Col1 | Riga1Col2 |

      Testo che segue la tabella.

    - Se la tabella originale è troppo ambigua o complessa, usa liste o testo descrittivo. Se l'input ha già una tabella formattata in Markdown, preservala assicurandoti comunque che abbia le righe vuote attorno.
10. Formule/Equazioni/Notazioni Matematiche/Scientifiche: Vedi punto 14.
11. Formatta l'output finale in MARKDOWN: Usa due cancelletti seguiti da spazio per i titoli H2 (## H2), tre cancelletti seguiti da spazio per i titoli H3 (### H3), due asterischi attorno a una parola per il grassetto (**grassetto**), un asterisco attorno a una parola per il corsivo (*corsivo*), liste puntate con un asterisco iniziale seguito da spazio (* punto elenco), o liste numerate con numero punto spazio (1. punto elenco). Mantieni la gerarchia e l'indentazione originali come nel testo di input.
12. Note a piè di pagina: Parafrasa il contenuto mantenendo il riferimento (esempio: [1] Testo parafrasato.).
13. Indica testo illeggibile scrivendo esattamente [TESTO ILLEGGIBILE].
14. Gestione FORMULE e NOTAZIONI SCIENTIFICHE (SOLO LaTeX, NIENTE Backticks!):
    - NON modificare il contenuto interno delle formule.
    - Usa ESCLUSIVAMENTE i delimitatori LaTeX standard: $ ... $ per formule inline, $$ ... $$ per formule display.
    - IMPORTANTISSIMO: NON USARE MAI i backtick (`) per la matematica. I backtick sono SOLO per il codice (es. `var=5`). Usa sempre e solo $formula$ o $$formula$$.
    - Parafrasa solo il testo attorno alle formule, lasciando la formula e i suoi delimitatori $/$$ intatti.
    - Non inventare formule. Esempi corretti nel Markdown finale: La relazione $p^M = a - bx^M(T)$ è importante. La famosa equazione è $$E=mc^2$$.
15. Gestione Specifica Valuta Dollaro (IMPORTANTE): Se devi usare il simbolo dollaro USA per valuta (esempio: 120 dollari), USA SEMPRE una barra rovesciata seguita dal simbolo dollaro (\$) nel Markdown. Esempio: Il prezzo è 120\$.
16. Restrizioni Aggiuntive sull'Output:
    - NON generare MAI linee orizzontali di separazione (usando ---, ***, o ___).
    - NON fare MAI riferimento ai numeri di pagina o alla struttura del documento originale nel testo parafrasato.
17. Output Finale: Produci UNICAMENTE il testo Markdown risultante dalla parafrasi. Deve contenere: placeholders IMGP... (posizionati correttamente nel flusso), tabelle Markdown corrette e spaziate con righe vuote prima e dopo, formule LaTeX corrette ($ o $$, NO backticks), valuta \$ corretta. NON includere: titoli aggiuntivi, header, footer, commenti, linee orizzontali, o altro testo non richiesto.
18. Scrivi nella lingua del testo originale.
"""

# --- 5. Funzioni di Utilità ---
# (Invariate)
def extract_text_images_annotations(pdf_content_bytes, temp_image_folder):
    """Estrae testo/immagini PDF usando l'analisi del layout (get_text("dict"))."""
    page_texts = []; image_references = {}; doc = None
    print("Avvio estrazione testo/immagini PDF con analisi layout (get_text dict)...")
    try:
        if not os.path.exists(temp_image_folder): os.makedirs(temp_image_folder); print(f"Cartella temporanea creata: {temp_image_folder}")
        doc = fitz.open(stream=pdf_content_bytes, filetype="pdf"); n_pages = doc.page_count; print(f"PDF caricato. Pagine: {n_pages}"); image_counter_total = 0
        for i in range(n_pages):
            page_num_actual = i + 1; page = doc.load_page(i)
            current_page_content_parts = [] ; img_count_on_page = 0
            try:
                page_dict = page.get_text("dict", sort=True) # Rimosso flags
                for block in page_dict["blocks"]:
                    if block["type"] == 0: # Blocco di testo
                        block_text = "";
                        for line in block["lines"]:
                            line_text = "";
                            for span in line["spans"]: line_text += span["text"]
                            block_text += line_text + " "
                        cleaned_block_text = unicodedata.normalize('NFC', block_text.strip());
                        if cleaned_block_text: current_page_content_parts.append(cleaned_block_text)
                    elif block["type"] == 1: # Blocco immagine
                        try:
                            img_count_on_page += 1; image_counter_total += 1; placeholder = f"IMGP{page_num_actual}xI{img_count_on_page}"; img_filename = f"{placeholder}.png"
                            local_image_path = os.path.join(temp_image_folder, img_filename); image_bytes = block["image"]
                            try:
                                pil_image = Image.open(io.BytesIO(image_bytes));
                                if pil_image.mode == 'P': pil_image = pil_image.convert('RGBA')
                                if pil_image.mode != 'RGB': pil_image = pil_image.convert('RGB')
                                pil_image.save(local_image_path, "PNG"); image_references[placeholder] = {'path': local_image_path, 'filename': img_filename}; current_page_content_parts.append(placeholder)
                            except Exception as save_e: print(f"     ! Warn: Salvataggio/Processo PIL img dict Pag {page_num_actual}, Blocco {block['number']}: {save_e}. IGNORATA.")
                        except Exception as img_extract_e: print(f"     ! Warn: Errore estrazione/dati immagine Blocco {block.get('number', 'N/A')} Pag {page_num_actual}: {img_extract_e}")
                page_final_text = "\n\n".join(current_page_content_parts); page_texts.append(page_final_text)
            except Exception as dict_err:
                print(f"Errore elaborazione blocchi (get_text dict) Pag {page_num_actual}: {dict_err}. Fallback a testo semplice.");
                try:
                    fallback_text = unicodedata.normalize('NFC', page.get_text("text", sort=True) or "")
                    page_texts.append(fallback_text.strip() + "\n\n[ERRORE_ESTRAZIONE_LAYOUT_PAGINA - Immagini Mancanti]\n\n"); print(f"     Fallback OK Pag {page_num_actual}. ATTENZIONE: Immagini perse.")
                except Exception as fallback_err: print(f"Errore anche nel fallback get_text Pag {page_num_actual}: {fallback_err}"); page_texts.append(f"\n\n[ERRORE_ESTRAZIONE_TOTALE_PAGINA_{page_num_actual}]\n\n")
        print(f"Estrazione PDF con analisi layout completata. Img temp referenziate: {len(image_references)}.")
        return page_texts, image_references
    except fitz.fitz.FileDataError as pdf_err: print(f"Errore Fatale Apertura PDF: {pdf_err}"); traceback.print_exc(); return None, None
    except Exception as e: print(f"Errore Fatale Estrazione PDF: {e}"); traceback.print_exc(); return None, None
    finally:
        if doc: doc.close(); print("Documento PyMuPDF chiuso.")

def paraphrase_text_chunk(text_chunk, instructions, model, chunk_header):
    """Invia chunk a Gemini per parafrasi."""
    if not text_chunk.strip(): print(f"Chunk '{chunk_header}' vuoto, saltato."); return ""
    prompt = f"""{instructions}

--- INIZIO TESTO DA PARAFRASARE ({chunk_header}) ---
{text_chunk}
--- FINE TESTO DA PARAFRASARE ({chunk_header}) ---

Fornisci SOLO il testo parafrasato in MARKDOWN per '{chunk_header}', seguendo TUTTE le istruzioni aggiornate (mantieni `IMGP...` NELLA LORO POSIZIONE ESATTA, formatta tabelle **CON RIGHE VUOTE PRIMA E DOPO**, formule LaTeX `$..$` o `$$..$$` SENZA USARE MAI backtick \` per la matematica, dollari valuta come `\$`, NON usare linee orizzontali ---, NON riferirti a numeri pagina originali). **NON INCLUDERE commenti, header, footer o altro testo al di fuori del Markdown risultante.**"""
    max_retries = 3; base_delay = 5
    for attempt in range(max_retries):
        print(f"      Tentativo {attempt + 1}/{max_retries} invio chunk '{chunk_header}' a Gemini ({model_name_in_use})...")
        paraphrased_text = None; final_text = ""; finish_reason_name = "UNKNOWN"; safety_ratings_str = "N/A"; candidate = None
        try:
            response = model.generate_content(prompt); block_reason=None
            if response.prompt_feedback: safety_ratings_str=str(response.prompt_feedback.safety_ratings) if response.prompt_feedback.safety_ratings else"N/A";block_reason=str(response.prompt_feedback.block_reason) if response.prompt_feedback.block_reason else None
            if block_reason: print(f"ATTENZIONE: Blocco PROMPT '{chunk_header}'(T{attempt+1}):{block_reason}. Sicurezza:{safety_ratings_str}")
            if not response.candidates: print(f"ATTENZIONE: No candidati '{chunk_header}'(T{attempt+1}). Sicurezza Prompt:{safety_ratings_str}")
            else:
                candidate=response.candidates[0]
                if candidate.safety_ratings:resp_safety_str=str(candidate.safety_ratings);safety_ratings_str=f"{safety_ratings_str}|R:{resp_safety_str}" if safety_ratings_str!="N/A" else f"R:{resp_safety_str}"
                finish_reason_name=candidate.finish_reason.name if candidate.finish_reason else "UNKNOWN"
                if finish_reason_name=="STOP":paraphrased_text=response.text
                elif finish_reason_name=="MAX_TOKENS":paraphrased_text=response.text;print(f"ATTENZIONE:MAX_TOKENS '{chunk_header}'(T{attempt+1}). Output potrebbe essere incompleto.")
                elif finish_reason_name=="SAFETY":print(f"ATTENZIONE:Blocco SAFETY RISPOSTA '{chunk_header}'(T{attempt+1}).Sicurezza:{safety_ratings_str}");paraphrased_text=None;
                elif finish_reason_name in["RECITATION","OTHER"]:print(f"ATTENZIONE:Terminazione anomala '{chunk_header}'(T{attempt+1}):{finish_reason_name}.Sicurezza:{safety_ratings_str}");paraphrased_text=None;
                else:print(f"ATTENZIONE:Terminazione non gestita '{chunk_header}'(T{attempt+1}):{finish_reason_name}.Sicurezza:{safety_ratings_str}");paraphrased_text=None;
            if paraphrased_text is not None:
                cleaned_text=paraphrased_text.strip();lines=cleaned_text.splitlines();
                lines_filtered=[l for l in lines if l.strip() and not l.strip().startswith("--- INIZIO") and not l.strip().startswith("--- FINE") and not l.strip() == f"--- {chunk_header} ---" and not re.match(r"^\s*[`]*(markdown)?\s*$", l, re.IGNORECASE) and not re.match(r"^\s*([-*_]\s*){3,}\s*$", l)]
                if lines_filtered and lines_filtered[-1].strip()=="```":lines_filtered.pop();
                if lines_filtered and lines_filtered[0].strip()=="```":lines_filtered.pop(0);
                final_text="\n".join(lines_filtered).strip();
                if not final_text and finish_reason_name=="STOP":print(f"      Info/Warn:Risposta '{chunk_header}' valida(STOP)ma vuota post-pulizia.")
            should_retry=(finish_reason_name in["SAFETY","RECITATION","OTHER"]or not candidate or block_reason)
            if(final_text or(paraphrased_text is not None and finish_reason_name=="MAX_TOKENS")or(paraphrased_text==""and finish_reason_name=="STOP"))and not should_retry:
                status_msg=f"(Nota:Troncato? {finish_reason_name})"if finish_reason_name!='STOP'else"(Completato)";print(f"      Chunk '{chunk_header}' parafrasato(T{attempt+1}).{status_msg}");return final_text
            if attempt<max_retries-1:
                if should_retry: delay=base_delay*(2**attempt);print(f"        -> Retry ({block_reason or finish_reason_name or 'No candidati'}).Attesa {delay}s...");time.sleep(delay);continue
                else: print(f"ERRORE INTERNO? T{attempt+1} '{chunk_header}' non riuscito ma non retryable? Motivo:{finish_reason_name},BloccoP:{block_reason}. Tentativo Retry...");delay=base_delay*(2**attempt);time.sleep(delay);continue
            else: # Fallimento definitivo
                err_msg = f"ERRORE FINALE API/GENERAZIONE per {chunk_header} dopo {max_retries} tentativi.\n"; err_msg += f"- Ultimo Motivo: {finish_reason_name}\n"; err_msg += f"- Blocco Prompt: {block_reason}\n"; err_msg += f"- Dettagli Sicurezza: {safety_ratings_str}\n"
                if paraphrased_text is not None: err_msg += f"- Ultimo Output (potrebbe essere parziale):\n{paraphrased_text.strip()}\n"
                else: err_msg += "- Nessun output valido generato nell'ultimo tentativo.\n"
                print(f"ERRORE FINALE (dopo retries) per {chunk_header}.")
                return f"\n\n```error\n[ERRORE DI ELABORAZIONE: {chunk_header}]\n\n{err_msg}\n```\n\n"
        except Exception as e:
            print(f"ERRORE CATTURATO API/ELAB '{chunk_header}'(T{attempt+1}):{e}");
            if not isinstance(e,(genai.types.StopCandidateException,genai.types.BlockedPromptException,genai.types.generation_types.BrokenResponseError,genai.types.generation_types.IncompleteIterationError)):traceback.print_exc()
            if attempt==max_retries-1:
                error_final_msg = f"[ERRORE DI ELABORAZIONE (Eccezione): {chunk_header}]\n\nEccezione: {e}\n"; print(f"ERRORE ECCEZIONE FINALE (dopo retries) per '{chunk_header}'.")
                return f"\n\n```error\n{error_final_msg}\n```\n\n"
            else: delay=base_delay*(2**attempt);print(f"        -> Errore, ritento tra {delay}s...");time.sleep(delay)
    print(f"ERRORE IMPREVISTO Loop '{chunk_header}'.")
    return f"\n\n```error\n[ERRORE LOGICA SCRIPT: {chunk_header}]\nFalliti {max_retries} tentativi in modo imprevisto.\n```\n\n"

def filter_and_save_valid_images(image_references, temp_image_folder, final_drive_image_folder, black_threshold=10, white_threshold=245):
    """Analizza img temp DAL PDF, copia valide in Drive, restituisce dict img valide e set placeholder rimossi."""
    # (Invariato)
    print(f"\n--- Filtro Immagini Vuote/Nere (dal PDF) e Copia Valide ---"); print(f"Analisi da: {temp_image_folder}"); print(f"Copia valide in: {final_drive_image_folder}"); print(f"Soglie: Nero < {black_threshold}, Bianco > {white_threshold}")
    valid_image_refs = {}; removed_placeholders = set(); copied_count = 0; error_count = 0; blank_count = 0
    if not image_references: print("Nessuna immagine iniziale dal PDF da analizzare."); return {}, set()
    try: os.makedirs(final_drive_image_folder, exist_ok=True)
    except OSError as e: print(f"!!! ERRORE CRITICO: Creazione cartella img dest '{final_drive_image_folder}': {e}"); return {}, set(image_references.keys())
    for placeholder, img_data in image_references.items():
        temp_path = img_data.get('path'); filename = img_data.get('filename')
        if not temp_path or not filename or not os.path.isfile(temp_path): print(f"     ! Warn: Ref/file non valido per {placeholder} (path: {temp_path}). Rimossa."); removed_placeholders.add(placeholder); error_count += 1; continue
        is_blank = False
        try:
            with Image.open(temp_path) as img: grayscale_img = img.convert('L'); min_val, max_val = grayscale_img.getextrema(); is_blank = max_val < black_threshold or min_val > white_threshold
        except UnidentifiedImageError: print(f"     ! Warn: Impossibile aprire/identificare {filename}. Rimossa."); removed_placeholders.add(placeholder); error_count += 1; continue
        except Exception as e: print(f"     ! Warn: Errore analisi immagine {filename}: {e}. Rimossa."); removed_placeholders.add(placeholder); error_count += 1; continue
        if is_blank: print(f"     - Rilevata immagine vuota/quasi vuota (PDF): {filename}. Rimossa."); removed_placeholders.add(placeholder); blank_count += 1
        else:
            final_path = os.path.join(final_drive_image_folder, filename)
            try: shutil.copy2(temp_path, final_path); valid_image_refs[placeholder] = {'filename': filename}; copied_count += 1
            except Exception as e: print(f"     ! ERRORE copia img PDF {filename} in Drive: {e}. Rimossa."); removed_placeholders.add(placeholder); error_count += 1
    print(f"Filtro immagini PDF completato. Valide copiate: {copied_count}. Vuote rimosse: {blank_count}. Errori: {error_count}.")
    return valid_image_refs, removed_placeholders

def mount_drive():
    """Monta Google Drive."""
    # (Invariato)
    try: drive.mount('/content/drive', force_remount=True); print("Google Drive montato con successo."); return True
    except Exception as e: print(f"Errore Critico durante il montaggio di Google Drive: {e}"); return False

def add_space_around_tables(markdown_text):
    """Aggiunge spazio attorno alle tabelle Markdown."""
    # (Invariato)
    print("  Applicazione post-processing per spaziatura tabelle...")
    lines = markdown_text.split('\n')
    new_lines = []
    num_lines = len(lines)
    added_before = 0
    added_after = 0
    for i, line in enumerate(lines):
        stripped_line = line.strip()
        current_line_is_table = stripped_line.startswith('|') and stripped_line.endswith('|')
        prev_line_is_blank = (i == 0) or (lines[i-1].strip() == "")
        prev_line_is_table = (i > 0) and lines[i-1].strip().startswith('|') and lines[i-1].strip().endswith('|')
        if current_line_is_table and not prev_line_is_table and not prev_line_is_blank and i > 0:
            new_lines.append("")
            added_before += 1
        new_lines.append(line)
        next_line_is_blank = (i == num_lines - 1) or (lines[i+1].strip() == "")
        next_line_is_table = (i < num_lines - 1) and lines[i+1].strip().startswith('|') and lines[i+1].strip().endswith('|')
        if current_line_is_table and not next_line_is_table and not next_line_is_blank and i < num_lines - 1:
            new_lines.append("")
            added_after += 1
    if added_before > 0 or added_after > 0:
        print(f"  Aggiunte {added_before} righe vuote prima e {added_after} dopo le tabelle.")
        return "\n".join(new_lines)
    else:
        print("  Nessuna riga vuota aggiuntiva necessaria per le tabelle.")
        return markdown_text

# --- 6. Funzione Principale di Esecuzione ---
def run_full_process():
    """Flusso principale."""
    # (Setup iniziale invariato)
    print("=============================================")
    print(f"--- Avvio Processo Parafrasi PDF v4.13.2 - FIX TABLE SPACE POSTPROC + LAYOUT IMG POS ({os.path.basename(PERMANENT_HEADER_LOGO_SOURCE_PATH)}) ---")
    print("=============================================")
    CHUNK_SIZE_PAGES = 10
    TEMP_IMAGE_FOLDER = "/content/temp_pdf_parafrasi_images_v4_13"
    DRIVE_BASE_OUTPUT_FOLDER = "/content/drive/MyDrive/Appunti TopNotes/markdown"
    print(f"Percorso base di output impostato su: {DRIVE_BASE_OUTPUT_FOLDER}")
    print(f"Percorso Logo Header FISSO su Drive: {PERMANENT_HEADER_LOGO_SOURCE_PATH}")
    if os.path.exists(TEMP_IMAGE_FOLDER): print(f"Pulizia cartella temporanea precedente: {TEMP_IMAGE_FOLDER}");
    try: shutil.rmtree(TEMP_IMAGE_FOLDER);
    except Exception as e: print(f"Warn: Errore rimozione temp: {e}")
    try: os.makedirs(TEMP_IMAGE_FOLDER)
    except Exception as e: print(f"Errore Critico: Impossibile creare cartella temporanea '{TEMP_IMAGE_FOLDER}': {e}"); return
    print("\n--- 1. Montaggio Google Drive ---")
    if not mount_drive(): return
    timestamp = time.strftime("%Y%m%d_%H%M%S")
    uploaded_file_name_base = "PDF_Parafrasato"; drive_output_subfolder_base = DRIVE_BASE_OUTPUT_FOLDER
    uploaded_file_name = None; output_md_path = None; drive_output_subfolder = None
    drive_image_folder_relative = "Images"; drive_image_folder_absolute = None
    header_image_filename_dest = "logo.jpg"; header_image_copy_path_dest = None
    header_image_copied_successfully = False
    print("\n--- 3. Caricamento File PDF ---"); print("Seleziona e carica il file PDF...")
    uploaded_pdf_content = None
    try:
        uploaded = files.upload()
        if not uploaded: print("Nessun file PDF caricato. Processo interrotto."); return
        uploaded_file_name = list(uploaded.keys())[0]; uploaded_pdf_content = uploaded[uploaded_file_name]
        print(f"File PDF '{uploaded_file_name}' caricato ({len(uploaded_pdf_content)} bytes).")
        sanitized_file_name = re.sub(r'[\\/*?:"<>|]', "_", os.path.splitext(uploaded_file_name)[0])
        uploaded_file_name_base = f"{sanitized_file_name}_{timestamp}"
        drive_output_subfolder = os.path.join(drive_output_subfolder_base, uploaded_file_name_base)
        output_md_filename = f"{uploaded_file_name_base}_parafrasato.md"
        output_md_path = os.path.join(drive_output_subfolder, output_md_filename)
        drive_image_folder_absolute = os.path.join(drive_output_subfolder, drive_image_folder_relative)
        header_image_copy_path_dest = os.path.join(drive_image_folder_absolute, header_image_filename_dest)
        print(f"Cartella Output: {drive_output_subfolder}"); print(f"File Markdown: {output_md_path}"); print(f"Cartella Immagini: {drive_image_folder_absolute}"); print(f"Logo Destinazione: {header_image_copy_path_dest}")
        os.makedirs(drive_image_folder_absolute, exist_ok=True); print(f"Cartelle Drive create/verificate.")
    except OSError as e: print(f"ERRORE CRITICO Creazione Cartelle Output Drive: {e}"); return
    except Exception as e: print(f"ERRORE CRITICO Upload PDF / Setup Percorsi: {e}"); traceback.print_exc(); return
    print("\n--- 4. Estrazione Contenuto PDF (con Analisi Layout) ---")
    page_texts, initial_image_references = extract_text_images_annotations(uploaded_pdf_content, TEMP_IMAGE_FOLDER)
    if page_texts is None or initial_image_references is None : print("ERRORE CRITICO Estrazione PDF. Processo interrotto."); return
    print(f"Estrazione PDF con Analisi Layout completata. Pagine: {len(page_texts)}. Img PDF iniziali: {len(initial_image_references)}")
    print(f"\n--- 5. Parafrasi Testo con Gemini ({model_name_in_use}) ---")
    full_paraphrased_markdown_with_placeholders = ""; total_pages = len(page_texts)
    total_chunks = math.ceil(total_pages / CHUNK_SIZE_PAGES) if total_pages > 0 else 0
    print(f"Divisione in {total_chunks} chunk da {CHUNK_SIZE_PAGES} pagine (circa).")
    md_save_success = False; chunk_processing_errors = 0
    if total_chunks == 0 and not initial_image_references: print("PDF vuoto. MD conterrà solo header (logo)."); full_paraphrased_markdown_with_placeholders = ""
    elif total_chunks == 0 and initial_image_references: print("PDF solo immagini. MD conterrà header + placeholder img PDF."); full_paraphrased_markdown_with_placeholders = "\n\n".join(initial_image_references.keys()) + "\n\n"
    else:
        for i in range(0, total_pages, CHUNK_SIZE_PAGES):
            chunk_num=(i//CHUNK_SIZE_PAGES)+1; chunk_header = f"CHUNK_{chunk_num}"; print(f"\n  Processando {chunk_header}/{total_chunks}...")
            text_chunk_to_process = "\n\n".join(page_texts[i:min(i + CHUNK_SIZE_PAGES, total_pages)])
            paraphrased_chunk_md = paraphrase_text_chunk(text_chunk_to_process, PARAPHRASING_INSTRUCTIONS, model, chunk_header)
            if "```error" in paraphrased_chunk_md: chunk_processing_errors+=1; print(f"!!! Errore API {chunk_header}. Inserito nel MD.");
            full_paraphrased_markdown_with_placeholders += paraphrased_chunk_md
            if not "```error" in paraphrased_chunk_md and paraphrased_chunk_md.strip(): full_paraphrased_markdown_with_placeholders += "\n\n"
            if chunk_num < total_chunks: pause_duration = 1.5; print(f"      ...Pausa {pause_duration}s..."); time.sleep(pause_duration)
        print("\n--- Parafrasi chunk completata ---")
        if chunk_processing_errors > 0: print(f"ATTENZIONE: {chunk_processing_errors} errori API durante parafrasi chunk (vedere blocchi 'error' nel MD).")
        else: print("Parafrasi chunk completata senza errori API apparenti.")
    print("\n--- 6. Pulizia Formule LaTeX (Rimozione Backtick Errati) ---")
    cleaned_markdown_formulas = full_paraphrased_markdown_with_placeholders; count_backticks_removed = 0
    try:
        cleaned_markdown_formulas, count1 = re.subn(r'`(\$[^\$]+\$)`', r'\1', cleaned_markdown_formulas)
        cleaned_markdown_formulas, count2 = re.subn(r'`(\$\$.*?\$\$)`', r'\1', cleaned_markdown_formulas, flags=re.DOTALL)
        count_backticks_removed = count1 + count2
        if count_backticks_removed > 0: print(f"  Rimossi {count_backticks_removed} backtick singoli adiacenti a formule $/$$..."); full_paraphrased_markdown_with_placeholders = cleaned_markdown_formulas
        else: print("  Nessun backtick singolo direttamente adiacente a $/$$ trovato.")
    except Exception as regex_e: print(f"  WARN: Errore pulizia regex formule: {regex_e}.")
    print("\n--- 7. Pulizia Spaziatura Tabelle Markdown ---")
    try:
        full_paraphrased_markdown_with_placeholders = add_space_around_tables(full_paraphrased_markdown_with_placeholders)
    except Exception as table_space_e:
        print(f"  WARN: Errore durante l'aggiunta di spazio attorno alle tabelle: {table_space_e}. Si procede senza questa pulizia.")
    print("\n--- 8. Filtro Immagini PDF e Copia Logo Fisso su Drive ---")
    valid_image_refs, removed_placeholders = filter_and_save_valid_images(initial_image_references, TEMP_IMAGE_FOLDER, drive_image_folder_absolute)
    print(f"Placeholders img PDF validi: {len(valid_image_refs)}"); print(f"Placeholders img PDF rimossi: {len(removed_placeholders)}")
    print(f"  Tentativo copia Logo Header da: {PERMANENT_HEADER_LOGO_SOURCE_PATH}")
    if not PERMANENT_HEADER_LOGO_SOURCE_PATH: print("  WARN: Percorso logo fisso non configurato."); header_image_copied_successfully = False
    elif not os.path.exists(PERMANENT_HEADER_LOGO_SOURCE_PATH): print(f"  !!! ERRORE CRITICO: Logo SORGENTE non trovato: {PERMANENT_HEADER_LOGO_SOURCE_PATH}"); header_image_copied_successfully = False
    elif not header_image_copy_path_dest: print("  !!! ERRORE INTERNO: Percorso destinazione logo non definito."); header_image_copied_successfully = False
    else:
        try: shutil.copy2(PERMANENT_HEADER_LOGO_SOURCE_PATH, header_image_copy_path_dest); header_image_copied_successfully = True; print(f"  Logo Header copiato in: {header_image_copy_path_dest}")
        except Exception as e: print(f"  !!! ERRORE copia Logo Header: {e}"); header_image_copied_successfully = False


    # --- 9. Post-Elaborazione Markdown Immagini PDF ---
    # <<< INIZIO SEZIONE 9 MODIFICATA CON DEBUG LENGTH/SNIPPET >>>
    print("\n--- 9. Post-Elaborazione Markdown Immagini PDF (Sostituzione Placeholder) ---")
    final_markdown_content = full_paraphrased_markdown_with_placeholders
    placeholders_replaced_count = 0; placeholders_removed_count = 0
    num_valid_placeholders = len(valid_image_refs) if valid_image_refs else 0
    num_removed_placeholders_set = len(removed_placeholders) if removed_placeholders else 0

    start_time_step9 = time.time() # <<< Tempo inizio passo

    if valid_image_refs:
        print(f"  Sostituzione {num_valid_placeholders} placeholder img PDF validi...");

        # Diagnostica Ordinamento Validi (Invariata)
        print(f"    DEBUG: Tentativo di ordinare {num_valid_placeholders} chiavi placeholder valide...")
        sort_start_time = time.time()
        sorted_placeholders = None
        try:
            valid_keys = list(valid_image_refs.keys())
            print(f"    DEBUG: Lista chiavi creata (primo elemento: '{valid_keys[0] if valid_keys else 'N/A'}'). Inizio ordinamento...")
            sorted_placeholders = sorted(valid_keys, key=len, reverse=True)
            sort_end_time = time.time()
            print(f"    DEBUG: Ordinamento completato in {sort_end_time - sort_start_time:.4f} sec.")
            if sorted_placeholders:
                 print(f"    DEBUG: Primo placeholder ordinato: '{sorted_placeholders[0]}'")
            else:
                 print(f"    DEBUG: La lista ordinata è vuota.")
        except Exception as sort_err:
            sort_end_time = time.time()
            print(f"    !!!! ERRORE CRITICO durante ordinamento placeholder validi ({sort_end_time - sort_start_time:.4f} sec): {sort_err}")
            traceback.print_exc()
            sorted_placeholders = []

        # Ciclo Sostituzione Validi (con string.replace e nuova diagnostica)
        if not sorted_placeholders:
             print("    WARN: Nessun placeholder valido trovato dopo il tentativo di ordinamento. Salto il ciclo di sostituzione.")
        else:
            print(f"\n    DEBUG: Inizio ciclo 'for' per {len(sorted_placeholders)} placeholder validi ordinati...")
            loop_start_time = time.time()
            for i, placeholder in enumerate(sorted_placeholders):
                iter_start_time = time.time()
                if i % 10 == 0 or i == len(sorted_placeholders) - 1:
                     print(f"      Sostituzione valido {i+1}/{len(sorted_placeholders)}: Elaborazione '{placeholder}'...")

                img_filename = valid_image_refs[placeholder]['filename'];
                relative_img_path = f"./{drive_image_folder_relative}/{img_filename}";
                markdown_tag = f"![{placeholder}]({relative_img_path})"

                # <<< NUOVA DIAGNOSTICA PRIMA DEL REPLACE >>>
                print(f"        DEBUG: i={i}, placeholder='{placeholder}'") # <<< Stampa indice e placeholder attuale
                try:
                    # Trova la posizione approssimativa del placeholder per lo snippet
                    # Cerca solo se la stringa non è eccessivamente grande per evitare find() lenti
                    current_length = len(final_markdown_content)
                    print(f"        DEBUG: Lunghezza markdown prima: {current_length}")
                    if current_length < 10 * 1024 * 1024: # Limite arbitrario (es. 10MB) per find()
                        placeholder_pos = final_markdown_content.find(placeholder)
                        snippet_start = max(0, placeholder_pos - 30)
                        snippet_end = min(current_length, placeholder_pos + len(placeholder) + 30)
                        snippet = final_markdown_content[snippet_start:snippet_end].replace('\n', '\\n') # Mostra snippet, escape newlines
                        print(f"        DEBUG: Posizione trovata: {placeholder_pos}")
                        print(f"        DEBUG: Snippet attorno: ...{snippet}...")
                    else:
                        print(f"        DEBUG: Lunghezza markdown ({current_length}) troppo grande, salto ricerca snippet.")
                except Exception as find_err:
                    print(f"        WARN: Errore nel trovare/stampare snippet per '{placeholder}': {find_err}")
                # <<< FINE NUOVA DIAGNOSTICA >>>


                replace_start_time = time.time()
                initial_len = len(final_markdown_content) # Ricalcola qui per sicurezza
                try:
                    print(f"        DEBUG: Tentativo di .replace()...") # <<< ULTIMA STAMPA PRIMA DEL POTENZIALE BLOCCO
                    # Usa string.replace per sostituzione letterale
                    final_markdown_content = final_markdown_content.replace(placeholder, markdown_tag)
                    print(f"        DEBUG: .replace() completato.") # <<< STAMPA DOPO IL POTENZIALE BLOCCO

                    replaced_in_iter = initial_len != len(final_markdown_content)

                    if replaced_in_iter:
                        placeholders_replaced_count += 1

                    replace_end_time = time.time()
                    print(f"        -> Sostituito '{placeholder}' in {replace_end_time - replace_start_time:.4f} sec.")

                except Exception as replace_err:
                     replace_end_time = time.time()
                     print(f"      !!!! ERRORE durante string.replace per '{placeholder}' ({replace_end_time - replace_start_time:.4f} sec): {replace_err}")
                     traceback.print_exc()

                iter_end_time = time.time()
                # Riduci la soglia LENTA per vedere se i tempi aumentano gradualmente
                if iter_end_time - iter_start_time > 5:
                    print(f"        -> Iterazione {i+1} ('{placeholder}') LENTA: {iter_end_time - iter_start_time:.2f} sec")

            loop_end_time = time.time()
            print(f"  Sostituiti {placeholders_replaced_count} placeholder img PDF validi. (Tempo ciclo sostituzione: {loop_end_time - loop_start_time:.2f} sec)")
    else: print("  Nessun placeholder img PDF valido da sostituire.")


    # --- Ciclo per i placeholder rimossi (con diagnostica simile ma usa ancora re.subn) ---
    if removed_placeholders:
        print(f"\n  Rimozione {num_removed_placeholders_set} placeholder img PDF non validi...");

        # Diagnostica Ordinamento Rimossi
        print(f"    DEBUG: Tentativo di ordinare {num_removed_placeholders_set} placeholder rimossi...")
        sort_start_time_rm = time.time()
        placeholders_to_remove_sorted = None
        try:
            removed_keys = list(removed_placeholders)
            print(f"    DEBUG: Lista chiavi rimossi creata (primo elemento: '{removed_keys[0] if removed_keys else 'N/A'}'). Inizio ordinamento...")
            placeholders_to_remove_sorted = sorted(removed_keys, key=len, reverse=True)
            sort_end_time_rm = time.time()
            print(f"    DEBUG: Ordinamento rimossi completato in {sort_end_time_rm - sort_start_time_rm:.4f} sec.")
            if placeholders_to_remove_sorted:
                 print(f"    DEBUG: Primo placeholder rimosso ordinato: '{placeholders_to_remove_sorted[0]}'")
            else:
                 print(f"    DEBUG: La lista rimossi ordinata è vuota.")
        except Exception as sort_err_removed:
            sort_end_time_rm = time.time()
            print(f"    !!!! ERRORE CRITICO durante ordinamento placeholder rimossi ({sort_end_time_rm - sort_start_time_rm:.4f} sec): {sort_err_removed}")
            traceback.print_exc()
            placeholders_to_remove_sorted = []

        # Ciclo Rimozione Rimossi
        if not placeholders_to_remove_sorted:
            print("    WARN: Nessun placeholder rimosso trovato dopo il tentativo di ordinamento. Salto il ciclo di rimozione.")
        else:
            print(f"\n    DEBUG: Inizio ciclo 'for' per {len(placeholders_to_remove_sorted)} placeholder rimossi ordinati...")
            loop_start_time_rm = time.time()
            for i, placeholder in enumerate(placeholders_to_remove_sorted):
                iter_start_time = time.time()
                if i % 10 == 0 or i == len(placeholders_to_remove_sorted) - 1:
                     print(f"      Rimozione non valido {i+1}/{len(placeholders_to_remove_sorted)}: Elaborazione '{placeholder}'...")

                pattern_placeholder_only = r'\s*' + re.escape(placeholder) + r'\s*\n?'

                # <<< Diagnostica prima della rimozione >>>
                print(f"        DEBUG: i={i}, placeholder='{placeholder}' (rimozione)")
                current_length = len(final_markdown_content)
                print(f"        DEBUG: Lunghezza markdown prima: {current_length}")
                # <<< Fine diagnostica >>>

                removal_start_time = time.time()
                try:
                     print(f"        DEBUG: Tentativo di re.subn() per rimozione...") # <<< PRIMA DEL BLOCCO?
                     final_markdown_content, count = re.subn(pattern_placeholder_only, '', final_markdown_content, flags=re.IGNORECASE | re.MULTILINE)
                     print(f"        DEBUG: re.subn() per rimozione completato.") # <<< DOPO IL BLOCCO?

                     if count > 0: placeholders_removed_count += count
                     removal_end_time = time.time()
                     print(f"        -> Rimosso '{placeholder}' (trovato {count} volte) in {removal_end_time - removal_start_time:.4f} sec.")

                except Exception as sub_err_removed:
                     removal_end_time = time.time()
                     print(f"      !!!! ERRORE durante re.subn per rimozione '{placeholder}' ({removal_end_time - removal_start_time:.4f} sec): {sub_err_removed}")
                     traceback.print_exc()


                iter_end_time = time.time()
                if iter_end_time - iter_start_time > 5:
                    print(f"        -> Iterazione rimozione {i+1} ('{placeholder}') LENTA: {iter_end_time - iter_start_time:.2f} sec")

            loop_end_time_rm = time.time()
            print(f"  Rimossi {placeholders_removed_count} placeholder img PDF non validi. (Tempo ciclo rimozione: {loop_end_time_rm - loop_start_time_rm:.2f} sec)")
    else: print("  Nessun placeholder img PDF non valido da rimuovere.")


    end_time_step9 = time.time() # <<< Tempo fine passo
    print(f"\n--- Fine Post-Elaborazione Markdown Immagini PDF (Tempo totale passo 9: {end_time_step9 - start_time_step9:.2f} sec) ---") # <<< Stampa tempo totale
    # <<< FINE SEZIONE 9 MODIFICATA >>>


    # --- 10. Composizione Finale (Solo Logo Header) ---
    # (Invariato)
    print("\n--- 10. Composizione Header finale (Solo Logo Fisso)...")
    header_content = ""
    if header_image_copied_successfully and header_image_copy_path_dest:
        header_image_relative_path = f"./{drive_image_folder_relative}/{header_image_filename_dest}"; header_content += f"![Logo Header]({header_image_relative_path})\n\n"; print(f"  Aggiunto tag per Logo Header: {header_image_relative_path}")
    else: print("  Logo Header non copiato o errore, non aggiunto al MD.")
    final_markdown_output = header_content + final_markdown_content.strip()
    print("\n--- Controllo Finale Output MD ---"); print(f"Lunghezza finale MD: {len(final_markdown_output)} caratteri."); print("------------------------------------\n")

    # --- 11. Salvataggio File Markdown Finale su Drive ---
    # (Invariato)
    print("\n--- 11. Salvataggio File Markdown Finale su Drive ---")
    if output_md_path:
        try:
            os.makedirs(os.path.dirname(output_md_path), exist_ok=True)
            with open(output_md_path, 'w', encoding='utf-8') as f: f.write(final_markdown_output)
            print(f"File Markdown finale salvato con successo in: {output_md_path}"); md_save_success = True
        except Exception as e: print(f"!!! ERRORE CRITICO salvataggio MD finale: {e}"); traceback.print_exc(); md_save_success = False
    else: print("Errore: Percorso MD output non definito."); md_save_success = False

    # --- 12. Pulizia Cartella Temporanea ---
    # (Invariato)
    print("\n--- 12. Pulizia ---")
    if os.path.exists(TEMP_IMAGE_FOLDER):
        print(f"Rimozione cartella temporanea: {TEMP_IMAGE_FOLDER}");
        try: shutil.rmtree(TEMP_IMAGE_FOLDER); print("Cartella temporanea rimossa.")
        except Exception as e: print(f"Warn: Errore rimozione temp: {e}")
    else: print("Cartella temporanea non trovata.")

    # --- Riepilogo Finale ---
    # (Invariato)
    print("\n=============================================")
    final_outcome = "ERRORE";
    if md_save_success: final_outcome = "COMPLETATO CON SUCCESSO"
    else: final_outcome = "TERMINATO CON ERRORI (Salvataggio MD Fallito/Non Tentato)"
    if chunk_processing_errors > 0 and md_save_success: final_outcome = "COMPLETATO (ma con errori API nei chunk - controllare MD)"
    if not header_image_copied_successfully and os.path.exists(PERMANENT_HEADER_LOGO_SOURCE_PATH) and md_save_success: final_outcome = "COMPLETATO (ma con ERRORE COPIA LOGO FISSO)"
    elif not header_image_copied_successfully and not os.path.exists(PERMANENT_HEADER_LOGO_SOURCE_PATH) and md_save_success: final_outcome = "COMPLETATO (ma ATTENZIONE: LOGO FISSO SORGENTE non trovato)"

    print(f"--- PROCESSO {final_outcome} ---")
    if output_md_path and md_save_success and os.path.isfile(output_md_path): print(f"File Markdown salvato su Drive in: {output_md_path}")
    elif output_md_path and md_save_success: print(f"ATTENZIONE: Salvataggio MD reportato OK, ma file non trovato in: {output_md_path}")
    elif output_md_path: print(f"Errore: File Markdown NON salvato. Doveva essere in: {output_md_path}")
    else: print("Errore: Percorso file Markdown non definito.")

    final_pdf_image_count = -1; logo_found_in_dest = False
    if drive_image_folder_absolute and os.path.isdir(drive_image_folder_absolute):
        try:
            all_files = [f for f in os.listdir(drive_image_folder_absolute) if os.path.isfile(os.path.join(drive_image_folder_absolute, f))]
            pdf_images = [f for f in all_files if f.lower().startswith('imgp') and f.lower().endswith('.png')]
            final_pdf_image_count = len(pdf_images); logo_found_in_dest = header_image_filename_dest in all_files
            print(f"Cartella Immagini: {drive_image_folder_absolute} ({final_pdf_image_count} img PDF, Logo trovato: {logo_found_in_dest})")
            if not header_image_copied_successfully and not logo_found_in_dest: print(f"  (Logo header non copiato/trovato - controllare sorgente: {PERMANENT_HEADER_LOGO_SOURCE_PATH})")
        except Exception as list_e: print(f"Cartella Immagini creata ({drive_image_folder_absolute}), ma errore nel listare i file: {list_e}")
    elif len(initial_image_references) > 0 : print(f"Errore: C'erano immagini PDF ma cartella immagini Drive ({drive_image_folder_absolute}) non trovata.")
    else: print("Nessuna immagine PDF è stata processata.")

    if not md_save_success or chunk_processing_errors > 0 or not header_image_copied_successfully:
        print("\nATTENZIONE: Il processo è terminato con errori o avvisi.")
        if not header_image_copied_successfully and not os.path.exists(PERMANENT_HEADER_LOGO_SOURCE_PATH): print(f"  >> CONTROLLA CHE IL FILE LOGO ESISTA IN: {PERMANENT_HEADER_LOGO_SOURCE_PATH} <<")
        elif not header_image_copied_successfully: print("  >> CONTROLLA I LOG PER ERRORI DURANTE LA COPIA DEL LOGO <<")
        if chunk_processing_errors > 0: print("  >> CONTROLLA I BLOCCHI 'error' NEL FILE MARKDOWN FINALE <<")
        if drive_output_subfolder and os.path.exists(drive_output_subfolder): print(f"Controllare output parziale in: {drive_output_subfolder}")
        elif not drive_output_subfolder: print("Cartella output non creata.")
    print("=============================================")


# --- 13. Esecuzione ---
run_full_process()

API Key Gemini caricata con successo.
Modello richiesto: gemini-2.5-flash-preview-04-17
Configurazione generazione: {'temperature': 0.6, 'top_p': 0.95, 'top_k': 64, 'max_output_tokens': 60000, 'response_mime_type': 'text/plain'}
Tentativo inizializzazione modello: gemini-2.5-flash-preview-04-17
Modello 'gemini-2.5-flash-preview-04-17' inizializzato con successo.
--- Avvio Processo Parafrasi PDF v4.13.2 - FIX TABLE SPACE POSTPROC + LAYOUT IMG POS (logo.jpg) ---
Percorso base di output impostato su: /content/drive/MyDrive/Appunti TopNotes/markdown
Percorso Logo Header FISSO su Drive: /content/drive/MyDrive/Appunti TopNotes/logo/logo.jpg
Warn: Errore rimozione temp: [Errno 2] No such file or directory: '/content/temp_pdf_parafrasi_images_v4_13'

--- 1. Montaggio Google Drive ---
Mounted at /content/drive
Google Drive montato con successo.

--- 3. Caricamento File PDF ---
Seleziona e carica il file PDF...


Saving R-Economia e gestione della banca - Borroni.pdf to R-Economia e gestione della banca - Borroni.pdf
File PDF 'R-Economia e gestione della banca - Borroni.pdf' caricato (4875430 bytes).
Cartella Output: /content/drive/MyDrive/Appunti TopNotes/markdown/R-Economia e gestione della banca - Borroni_20250421_173651
File Markdown: /content/drive/MyDrive/Appunti TopNotes/markdown/R-Economia e gestione della banca - Borroni_20250421_173651/R-Economia e gestione della banca - Borroni_20250421_173651_parafrasato.md
Cartella Immagini: /content/drive/MyDrive/Appunti TopNotes/markdown/R-Economia e gestione della banca - Borroni_20250421_173651/Images
Logo Destinazione: /content/drive/MyDrive/Appunti TopNotes/markdown/R-Economia e gestione della banca - Borroni_20250421_173651/Images/logo.jpg
Cartelle Drive create/verificate.

--- 4. Estrazione Contenuto PDF (con Analisi Layout) ---
Avvio estrazione testo/immagini PDF con analisi layout (get_text dict)...
PDF caricato. Pagine: 149
Estrazione PD